In [50]:
from chrome_crawler import Chrome_Crawler
from lib import *
from person import *

In [45]:
PERSON_XPATH = {
    'name':'/html[1]/body[1]/div[1]/div[3]/div[2]/table[2]/tbody[1]/tr[1]/td[1]/h1[1]',
    'movie': {
        'date'          :'/html[1]/body[1]/div[1]/div[3]/div[2]/table[2]/tbody[1]/tr[1]/td[1]/table[1]/tbody[1]/tr/td[1]/font[1]',
        'title'         :'/html[1]/body[1]/div[1]/div[3]/div[2]/table[2]/tbody[1]/tr[1]/td[1]/table[1]/tbody[1]/tr/td[2]/font[1]/a[1]',
        'studio'        :'/html[1]/body[1]/div[1]/div[3]/div[2]/table[2]/tbody[1]/tr[1]/td[1]/table[1]/tbody[1]/tr/td[3]/font[1]/a[1]',
        'lifetimeGross' :'/html[1]/body[1]/div[1]/div[3]/div[2]/table[2]/tbody[1]/tr[1]/td[1]/table[1]/tbody[1]/tr/td[4]/font[1]',
        'opening'       :'/html[1]/body[1]/div[1]/div[3]/div[2]/table[2]/tbody[1]/tr[1]/td[1]/table[1]/tbody[1]/tr/td[6]/font[1]',
    },
    'average':'/html[1]/body[1]/div[1]/div[3]/div[2]/table[2]/tbody[1]/tr[1]/td[1]/font[2]/b[1]'
}

In [46]:
_chrome_option = webdriver.ChromeOptions()
_chrome_option.add_argument("headless")
chrome_option = _chrome_option
driver = webdriver.Chrome(chrome_options=chrome_option)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [47]:
def get_list_person():
    # Get json movie list from folder Movies
    listMovies = os.listdir('Movies')
    listPeople = []

    for movie in listMovies:
        # Verify movie is .json file
        if (not(movie.endswith('.json'))):
            break

        # Create link of movie
        url = 'Movies\\%s' % movie
        # Get movie information
        movieInfo = json.load(open(url, 'r'))
        # Get list persons from movieInfo
        listPeople += movieInfo['director'] + movieInfo['writer'] + movieInfo['actor'] + movieInfo['producer']

    # Remove duplicated values
    people = np.unique(listPeople)
    return people

In [48]:
def get_person_info(url):
    driver.get(url)
    # driver.get(linkGenre)

    id = url.split('id=')[1][:-4]
    role = url.split('view=')[1].split('&')[0]

    name = driver.find_element_by_xpath(PERSON_XPATH['name']).text
    average = driver.find_element_by_xpath(PERSON_XPATH['average']).text.split('$')[1]

    elements = driver.find_elements_by_xpath(PERSON_XPATH['movie']['date'])[1:]
    date = [e.text for e in elements]


    elements = driver.find_elements_by_xpath(PERSON_XPATH['movie']['title'])[1:]
    title = [e.get_attribute('href')[41:-4] for e in elements]


    elements = driver.find_elements_by_xpath(PERSON_XPATH['movie']['studio'])[1:]
    studio = [e.get_attribute('href')[51:-4] for e in elements]

    elements = driver.find_elements_by_xpath(PERSON_XPATH['movie']['lifetimeGross'])[1:]
    lifetimeGross = [e.text[1:].replace(',','') for e in elements]

    elements = driver.find_elements_by_xpath(PERSON_XPATH['movie']['opening'])[1:]
    opening = [e.text[1:].replace(',','') for e in elements]
    opening


    movies = pd.DataFrame({
        'date': date,
        'title': title,
        'studio': studio,
        'lifetimeGross': lifetimeGross,
        'opening': opening
    }).to_dict('records')

    person = {
        'id':id,
        'name':name,
        'role':role,
        'average':average,
        'movies':movies
    }

    return person


In [49]:
person_crawler = Person_Crawler()

persons = get_list_person()
len(np.unique(persons))

735

In [6]:
_round = 0

while _round < len(persons):
    person = get_person_info(persons[_round])
    id = person['id']
    try:
        with open('Persons/%s.json' % id, 'w') as fp:
            json.dump(person, fp)

        time.sleep(np.random.randint(1,4)/4)

        # Reset driver:
        if _round % 50 == 0:
            print ("Crawling person...\t%d/%d\n"%(_round, len(persons)))
            person_crawler.restart_crawler(delay_time=15)
            
        _round += 1
    except:
        print('exception: %s' % id)
        with open('PersonExceptions.txt', 'a+') as f_except:
            f_except.write(id + '\n')

Crawling person...	0/735

Crawling person...	50/735

Crawling person...	100/735

Crawling person...	150/735

Crawling person...	200/735

Crawling person...	250/735

Crawling person...	300/735

Crawling person...	350/735

Crawling person...	400/735

Crawling person...	450/735

Crawling person...	500/735

Crawling person...	550/735

Crawling person...	600/735

Crawling person...	650/735

Crawling person...	700/735

